# [ 3-4. 시설관리비 설정 ]

## 1. 필요한 모듈 import 및 기본 설정
### 1-1. 필요한 모듈 import

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

from m00_general_function import subtotal
from m01_assumption import assumption
from m02_index import index
from m04_operating_income import operating_income

### 1-2. assumption 데이터 확인

In [2]:
assumption['facility_cost']

{'통상수선비': {'TypeA': 1000, 'TypeB': 1200, 'TypeC': 2000},
 '대수선공사비': {'TypeA': 30000000, 'TypeB': 35000000, 'TypeC': 50000000},
 '수선시작일': {'TypeA': datetime.datetime(2025, 3, 1, 0, 0),
  'TypeB': datetime.datetime(2025, 4, 1, 0, 0),
  'TypeC': datetime.datetime(2025, 5, 1, 0, 0)},
 '수선종료일': {'TypeA': datetime.datetime(2025, 3, 31, 0, 0),
  'TypeB': datetime.datetime(2025, 4, 30, 0, 0),
  'TypeC': datetime.datetime(2025, 5, 31, 0, 0)}}

### 1-3. 빈 facility_cost 딕셔너리 설정

In [3]:
facility_cost = {}

## 2. 통상수선비

In [4]:
data = []
for dt in index['model']:
    dct = {}

    if dt in index['operating']:
        dct['TypeA'] = int(
            operating_income['TypeA'].loc[dt, '판매객실수'] * 
            assumption['facility_cost']['통상수선비']['TypeA'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeB'] = int(
            operating_income['TypeB'].loc[dt, '판매객실수'] * 
            assumption['facility_cost']['통상수선비']['TypeB'] *
            index['연간인상률'].loc[dt, '운영비']
        )
        dct['TypeC'] = int(
            operating_income['TypeC'].loc[dt, '판매객실수'] * 
            assumption['facility_cost']['통상수선비']['TypeC'] *
            index['연간인상률'].loc[dt, '운영비']
        )
    else:
        dct['TypeA'] = 0
        dct['TypeB'] = 0
        dct['TypeC'] = 0
        
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

facility_cost['통상수선비'] = pd.DataFrame(data, index=index['model'])

## 3. 대수선공사비

In [5]:
data = []
for dt in index['model']:
    dct = {}
    if dt in index['수선TypeA']:
        dct['TypeA'] = int(
            assumption['business_overview']['객실수']['TypeA'] * 
            assumption['facility_cost']['대수선공사비']['TypeA']
        )
    else:
        dct['TypeA'] = 0

    if dt in index['수선TypeB']:
        dct['TypeB'] = int(
            assumption['business_overview']['객실수']['TypeB'] * 
            assumption['facility_cost']['대수선공사비']['TypeB']
        )
    else:
        dct['TypeB'] = 0

    if dt in index['수선TypeC']:
        dct['TypeC'] = int(
            assumption['business_overview']['객실수']['TypeC'] * 
            assumption['facility_cost']['대수선공사비']['TypeC']
        )
    else:
        dct['TypeC'] = 0
    
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

facility_cost['대수선공사비'] = pd.DataFrame(data, index=index['model'])

## 4. SubTotal

In [6]:
facility_cost['TypeA'] = subtotal(facility_cost, 'TypeA')
facility_cost['TypeB'] = subtotal(facility_cost, 'TypeB')
facility_cost['TypeC'] = subtotal(facility_cost, 'TypeC')
facility_cost['Total'] = subtotal(facility_cost, 'Total')

## 5. facility_cost.py 파일 작성

In [ ]:
# m06_facility_cost.py

import pandas as pd

from m00_general_function import subtotal
from m01_assumption import assumption
from m02_index import index
from m04_operating_income import operating_income


facility_cost = {}

#### 1. 통상수선비
data = []
for dt in index['model']:
    dct = {}
    dct['TypeA'] = int(
        operating_income['TypeA'].loc[dt, '판매객실수'] * 
        assumption['facility_cost']['통상수선비']['TypeA'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeB'] = int(
        operating_income['TypeB'].loc[dt, '판매객실수'] * 
        assumption['facility_cost']['통상수선비']['TypeB'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['TypeC'] = int(
        operating_income['TypeC'].loc[dt, '판매객실수'] * 
        assumption['facility_cost']['통상수선비']['TypeC'] *
        index['연간인상률'].loc[dt, '운영비']
    )
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

facility_cost['통상수선비'] = pd.DataFrame(data, index=index['model'])


#### 2. 대수선공사비
data = []
for dt in index['model']:
    dct = {}
    if dt in index['수선TypeA']:
        dct['TypeA'] = int(
            assumption['business_overview']['객실수']['TypeA'] * 
            assumption['facility_cost']['대수선공사비']['TypeA']
        )
    else:
        dct['TypeA'] = 0

    if dt in index['수선TypeB']:
        dct['TypeB'] = int(
            assumption['business_overview']['객실수']['TypeB'] * 
            assumption['facility_cost']['대수선공사비']['TypeB']
        )
    else:
        dct['TypeB'] = 0

    if dt in index['수선TypeC']:
        dct['TypeC'] = int(
            assumption['business_overview']['객실수']['TypeC'] * 
            assumption['facility_cost']['대수선공사비']['TypeC']
        )
    else:
        dct['TypeC'] = 0
    
    dct['Total'] = dct['TypeA'] + dct['TypeB'] + dct['TypeC']
    data.append(dct)

facility_cost['대수선공사비'] = pd.DataFrame(data, index=index['model'])


#### 3. SubTotal
facility_cost['TypeA'] = subtotal(facility_cost, 'TypeA')
facility_cost['TypeB'] = subtotal(facility_cost, 'TypeB')
facility_cost['TypeC'] = subtotal(facility_cost, 'TypeC')
facility_cost['Total'] = subtotal(facility_cost, 'Total')